<a href="https://colab.research.google.com/github/TeeshynaJ/-anime_forecasting/blob/main/anime_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#first we import libraries to use
import pandas as pd
from google.colab import auth
import pandas_gbq
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import random as random
from scipy import stats

# Will collect credentials
auth.authenticate_user()

In [ ]:
query = "SELECT * FROM `anime-forecasting.main_table_joined_not_exploaded.left_join_mega_table_exclude_eps_not_exploaded`"

In [ ]:
table = pandas_gbq.read_gbq(query, project_id="anime-forecasting")

Downloading: 100%|██████████|


#Machine Learning - Clustering

In [ ]:
#scalers
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
#encoders
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
#neighbors
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans

In [ ]:
#validating
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict

In [ ]:
table.drop_duplicates(subset='idMal',inplace=True)

In [ ]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2521 entries, 0 to 2537
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   idMal                      2521 non-null   Int64  
 1   title                      2521 non-null   object 
 2   format                     2521 non-null   object 
 3   episodes                   2521 non-null   Int64  
 4   duration                   2521 non-null   Int64  
 5   status                     2521 non-null   object 
 6   startDay                   2521 non-null   Int64  
 7   startMonth                 2521 non-null   Int64  
 8   startYear                  2521 non-null   Int64  
 9   meanScore                  2521 non-null   float64
 10  favourites                 2521 non-null   Int64  
 11  genres                     2521 non-null   object 
 12  scoreDistribution_score    2521 non-null   Int64  
 13  statusDistribution_score   2521 non-null   Int64  
 1

In [ ]:
table['idMal'].nunique()

2521

In [ ]:
table.columns

Index(['idMal', 'title', 'format', 'episodes', 'duration', 'status',
       'startDay', 'startMonth', 'startYear', 'meanScore', 'favourites',
       'genres', 'scoreDistribution_score', 'statusDistribution_score',
       'statusDistribution_status', 'rankings_context', 'tags', 'start_date',
       'external_sites', 'Score', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Popularity', 'Favorites', 'Scored_by'],
      dtype='object')

In [ ]:
table = table.drop(['format','startDay', 'startMonth', 'startYear','scoreDistribution_score', 'statusDistribution_score',
       'statusDistribution_status', 'rankings_context','start_date',
       'external_sites', 'Score', 'Premiered', 'Producers', 'Licensors',
       'Studios','Scored_by'], axis=1)

In [ ]:
table = table.drop(['Popularity', 'Favorites'], axis=1)

In [ ]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2521 entries, 0 to 2537
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   idMal       2521 non-null   Int64  
 1   title       2521 non-null   object 
 2   episodes    2521 non-null   Int64  
 3   duration    2521 non-null   Int64  
 4   status      2521 non-null   object 
 5   meanScore   2521 non-null   float64
 6   favourites  2521 non-null   Int64  
 7   genres      2521 non-null   object 
 8   tags        2521 non-null   object 
dtypes: Int64(4), float64(1), object(4)
memory usage: 206.8+ KB


In [ ]:
table['genres'] = table['genres'].apply(lambda x: x.strip('][').split(', '))

In [ ]:
table.explode('genres')

,idMal,title,episodes,duration,status,meanScore,favourites,genres,tags
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Action',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Adventure',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Fantasy',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,'Sci-Fi',"['Isekai', 'Anti-Hero', 'Crime', 'Crossover', ..."
1,35333,Hitori no Shita - The Outcast 2,24,24,FINISHED,71.0,218,'Action',"['Super Power', 'Urban Fantasy', 'Martial Arts..."
...,...,...,...,...,...,...,...,...,...
2536,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,'Adventure',['Historical']
2536,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,'Drama',['Historical']
2536,960,Legend of the Condor Hero II,26,20,FINISHED,68.0,6,'Romance',['Historical']
2537,50185,Salaryman's Club,12,23,FINISHED,72.0,338,'Drama',"['Badminton', 'Work', 'Primarily Male Cast', '..."


In [ ]:
from collections import Counter
genres_table = (table
 .set_index('idMal')
 .genres
 .map(Counter)
 .apply(pd.Series)
 .fillna(0, downcast='infer')
 .reset_index()
)

<ipython-input-33-3c1f32bd4bb0>:2: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  genres_table = (table


In [ ]:
table['tags'] = table['tags'].apply(lambda x: x.strip('][').split(', '))

In [ ]:
tags_table = (table
 .set_index('idMal')
 .tags
 .map(Counter)
 .apply(pd.Series)
 .fillna(0, downcast='infer')
 .reset_index()
)

<ipython-input-35-612219a1201d>:1: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  tags_table = (table


In [ ]:
genres_table

,idMal,'Action','Adventure','Fantasy','Sci-Fi','Supernatural','Drama','Mystery','Comedy','Sports','Horror','Ecchi','Romance','Psychological','Music','Mahou Shoujo','Slice of Life','Mecha','Thriller',
0,55848,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,35333,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,54492,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,43989,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,44863,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,257,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2517,1142,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
2518,16353,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2519,960,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
tags_table

,idMal,'Isekai','Anti-Hero','Crime','Crossover','Female Protagonist','Super Power','War','Magic','Primarily Adult Cast',...,'Irrumatio','Virginity','Jazz Music','Nakadashi','Anal Sex','Defloration','Flat Chest','Blackmail','Asphyxiation','Sex Toys'
0,55848,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,35333,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,54492,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,43989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44863,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,257,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2517,1142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,16353,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2519,960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tags_table.value_counts()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,count
idMal,'Isekai','Anti-Hero','Crime','Crossover','Female Protagonist','Super Power','War','Magic','Primarily Adult Cast','Conspiracy','Guns','Prison','Language Barrier','Shapeshifting','Meta','Swordplay','Primarily Male Cast','Henshin','Dragons','CGI','Necromancy','Skeleton','Elf','Heterosexual','Nudity','Urban Fantasy','Martial Arts','Male Protagonist','Wuxia','Medicine','Historical','Detective','Foreign','Politics','Ancient China','Drugs','Work','Royal Affairs','Adoption','Unrequited Love','Alchemy','Maids','Tragedy','Food','Slavery','Chibi','Asexual','Episodic','Go',,'Cars','Classic Literature','Motorcycles','Kids','Time Manipulation','Travel','Ships','Ghost','Primarily Child Cast','School','Youkai','Afterlife','Parody','Demons','Animals','Ensemble Cast','Tanned Skin','Found Family','Military','Robots','Aliens','Teacher','Seinen','Time Skip','Space','Environmental','Slapstick','Rural','Outdoor','Super Robot','Proxy Battle','Denpa','Dissociative Identities','Cosmic Horror','Rehabilitation','Cult','Mythology','Bullying','Philosophy','Love Triangle','Kuudere','Card Battle','Idol','Cute Girls Doing Cute Things','Amnesia','Gore','Survival','Body Horror','Dancing','Vampire','Coming of Age','Incest','Feet','Snowscape','Primarily Female Cast','Achronological Order','Gods','Primarily Teen Cast','Post-Apocalyptic','School Club','Memory Manipulation','Religion','Drawing','Photography','Filmmaking','Archery','Cute Boys Doing Cute Things','Tsundere','Twins','Anachronism','Bisexual','Video Games','Full CGI','Urban','Mopeds','Disability','Dungeon','Kemonomimi','Zombie','Veterinarian','Reincarnation','Anthology','Anthropomorphism','Shounen','Cultivation','Superhero','Gender Bending','Revenge','Cyborg','Dinosaurs','Body Swapping','Battle Royale','Police','Wrestling','Surreal Comedy','Dystopian','Real Robot','Shoujo','Tomboy','Tokusatsu','LGBTQ+ Themes','Age Gap','Yuri','Alternate Universe','Shrine Maiden','Makeup','Gambling','Volleyball','Female Harem','College','Death Game','Fairy Tale','Terrorism','MILF','Boarding School','Arranged Marriage','Band','Hip-hop Music','Bar','Rock Music','Fugitive','Monster Boy','Estranged Family','Kingdom Management','Primarily Animal Cast','Kaiju','Artificial Intelligence','Delinquents','Cyberpunk','Monster Girl','Yandere','Rape','Family Life','Coastal','Iyashikei','Mafia','Trains','Noir','Vore','Samurai','Cannibalism','Mermaid','Watersports','Bondage','Masochism','Tentacles','Gyaru','Chuunibyou','Musical Theater','Goblin','Rugby','Educational','Orphan','Ninja','Transgender','Clone','Lost Civilization','Pandemic','Angels','Aviation','Classical Music','Ice Skating','Athletics','Butler','Psychosexual','Board Game','Villainess','Josei','Ojou-sama','Shogi','Parkour','Achromatic','Homeless','Espionage','Suicide','Nun','Stop Motion','Otaku Culture','Astronomy',"""Boys' Love""",'Virtual World','Crossdressing','Restaurant','Torture','Time Loop','Witch','Assassins','Large Breasts','Fitness','Agriculture','Fairy','Nekomimi','Fake Relationship','Parenthood','Tennis','Football','Boxing','Femboy','Natural Disaster','Space Opera','Desert','Steampunk','Fashion','Pirates','Male Harem','Class Struggle','Matriarchy','Cheerleading','Age Regression','Werewolf','Satire','Baseball','Spearplay','Sadism','Poker','Lacrosse','Acting','Polyamorous','Chimera','Economics','Gangs','Succubus','Fishing','Aromantic','Writing','Calligraphy','Agender','Dullahan','Basketball','Mixed Gender Harem','Hikikomori','Cosplay','Yakuza','Circus','Inn','Advertisement','POV','Badminton','Marriage','Criminal Organization','Swimming','Human Pet','Tanks','Femdom','Rotoscoping','Rakugo','Masturb

In [ ]:
tags_remove = pd.DataFrame(tags_table.sum(axis=0))

In [ ]:
tags_remove

,0
idMal,60398653
'Isekai',263
'Anti-Hero',216
'Crime',165
'Crossover',16
...,...
'Defloration',1
'Flat Chest',1
'Blackmail',1
'Asphyxiation',1


In [ ]:
col_remove =list(tags_remove[tags_remove[0] < 6].index)

In [ ]:
tags_table=tags_table.drop(columns=col_remove)

In [ ]:
tags_table

,idMal,'Isekai','Anti-Hero','Crime','Crossover','Female Protagonist','Super Power','War','Magic','Primarily Adult Cast',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,55848,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,35333,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,54492,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,43989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44863,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,257,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2517,1142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,16353,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2519,960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
joined_table = pd.concat([table,genres_table,tags_table],axis=1)

In [ ]:
joined_table = joined_table.drop(['genres','tags'], axis=1)

In [ ]:
joined_table

,idMal,title,episodes,duration,status,meanScore,favourites,idMal,'Action','Adventure',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,55848,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35333,Hitori no Shita - The Outcast 2,24,24,FINISHED,71.0,218,35333,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,54492,The Apothecary Diaries,24,23,FINISHED,88.0,10978,54492,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,43989,Pleasant Goat and Big Big Wolf: Paddi the Amaz...,52,25,FINISHED,35.0,0,43989,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,44863,Magic Wonderland,26,22,FINISHED,51.0,0,44863,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,<NA>,NaN,<NA>,<NA>,NaN,NaN,<NA>,50346,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2154,<NA>,NaN,<NA>,<NA>,NaN,NaN,<NA>,38853,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2155,<NA>,NaN,<NA>,<NA>,NaN,NaN,<NA>,39535,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2468,<NA>,NaN,<NA>,<NA>,NaN,NaN,<NA>,48555,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model_table = joined_table.drop(['idMal','duration','title','status','favourites'], axis=1)

In [ ]:
new_table = table.groupby('idMal').count()

In [ ]:
new_table[new_table['title']>1]

,title,episodes,duration,status,meanScore,favourites,genres,tags
idMal,,,,,,,,


In [ ]:
model_table.isnull().sum()

,0
episodes,17
meanScore,17
'Action',17
'Adventure',17
'Fantasy',17
...,...
'VTuber',17
'Cycling',17
'Centaur',17
'American Football',17


In [ ]:
joined_table.isnull().sum()

,0
idMal,17
title,17
episodes,17
duration,17
status,17
...,...
'VTuber',17
'Cycling',17
'Centaur',17
'American Football',17


In [ ]:
new_table.isnull().sum()

,0
title,0
episodes,0
duration,0
status,0
meanScore,0
favourites,0
genres,0
tags,0


In [ ]:
model_table.dropna(inplace=True)

In [ ]:
joined_table.dropna(inplace=True)

In [ ]:
table.reset_index(inplace=True)

In [ ]:
table[table['idMal']== 2521]

,index,idMal,title,episodes,duration,status,meanScore,favourites,genres,tags
902,908,2521,Great Dangaioh,13,23,FINISHED,52.0,9,"['Action', 'Mecha', 'Sci-Fi']","['Robots', 'Male Protagonist', 'Female Protago..."


#Predicting

In [ ]:
model_table

,episodes,meanScore,'Action','Adventure','Fantasy','Sci-Fi','Supernatural','Drama','Mystery','Comedy',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,10,63.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24,71.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24,88.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52,35.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26,51.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,12,68.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2517,22,56.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2518,22,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2519,26,62.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
joined_table

,idMal,title,episodes,duration,status,meanScore,favourites,idMal,'Action','Adventure',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
0,55848,Suicide Squad ISEKAI,10,24,FINISHED,63.0,841,55848,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35333,Hitori no Shita - The Outcast 2,24,24,FINISHED,71.0,218,35333,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,54492,The Apothecary Diaries,24,23,FINISHED,88.0,10978,54492,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,43989,Pleasant Goat and Big Big Wolf: Paddi the Amaz...,52,25,FINISHED,35.0,0,43989,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,44863,Magic Wonderland,26,22,FINISHED,51.0,0,44863,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,56838,"Studio Apartment, Good Lighting, Angel Included",12,24,FINISHED,68.0,497,257,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2517,1537,Ryu the Primitive Boy,22,25,FINISHED,56.0,8,1142,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2518,2703,Super YoYo,22,23,FINISHED,61.0,11,16353,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2519,2501,Papuwa,26,25,FINISHED,62.0,5,960,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Define X and y
X = model_table #the genres and tags
y = model_table['meanScore'] #"meanScore", "Popularity", "favourites" -- possible y columns?

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

knn_model = KNeighborsRegressor().fit(X_scaled,y) # Instanciate and train model

In [ ]:
knn_stored = {}
knn_list = []

In [ ]:
def find_nearest(idMal):
    index = np.where(joined_table['idMal'] == idMal)[0][0]
    test_item = joined_table.iloc[index:(index+1)]
    X_new = test_item.drop(['idMal','duration','title','status','favourites'], axis=1)
    X_new_scaled = scaler.transform(X_new)
    ind_list = list(knn_model.kneighbors(X_new_scaled,n_neighbors=10)[1][0])[1:]
    ids = joined_table.iloc[ind_list]
    knn_list.append(ids)
    return ids


In [ ]:
new_joined_table = joined_table.loc[:, ~joined_table.columns.duplicated()]

In [ ]:
new_joined_table[new_joined_table['idMal'] == 38084]

,idMal,title,episodes,duration,status,meanScore,favourites,'Action','Adventure','Fantasy',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
879,38084,Fate/Grand Order Absolute Demonic Front: Babyl...,21,24,FINISHED,78.0,1808,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
find_nearest(38084) #20 - naruto, 1535 - death note

,idMal,title,episodes,duration,status,meanScore,favourites,idMal,'Action','Adventure',...,'Airsoft','Inseki','Augmented Reality','Karuta','Netorare','VTuber','Cycling','Centaur','American Football','Elderly Protagonist'
504,11837,Zetman,13,24,FINISHED,63.0,180,32608,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1655,50307,TONIKAWA: Over The Moon For You Season 2,12,24,FINISHED,76.0,1430,4415,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1061,52990,Our Dating Story: The Experienced You and The ...,12,24,FINISHED,66.0,1183,2826,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1698,50346,Call of the Night,13,23,FINISHED,79.0,6690,2980,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
468,1161,MAZE (TV),25,24,FINISHED,62.0,6,1005,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,31427,Zinba,52,22,FINISHED,63.0,11,31427,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1303,50204,Tokyo 24th Ward,12,24,FINISHED,61.0,194,56420,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1528,30895,Haruchika - Haruta & Chika,12,24,FINISHED,58.0,79,37576,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1854,25283,Sky Wizards Academy,12,24,FINISHED,58.0,347,51064,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
